In [1]:
import sys
sys.path.append('/content/drive/MyDrive/ColabNotebooks/CANN_Pietro/WorkspaceCANN')

In [2]:
from CANNLib.Datas.Datasets import *
from torch.utils.data import DataLoader

In [3]:
data = IsotropicIncompressibleDataset()

test_set = DataLoader(data,batch_size=4)

for batch in test_set:

  X,y = batch['Data'], batch['Psi']

In [4]:
X

{'C': tensor([[[2.2539, 2.3114, 2.7816],
          [2.3114, 2.7173, 2.7241],
          [2.7816, 2.7241, 3.4805]],
 
         [[2.1473, 2.1642, 2.9082],
          [2.1642, 3.0830, 3.6032],
          [2.9082, 3.6032, 4.4485]],
 
         [[2.7070, 3.4798, 2.4661],
          [3.4798, 4.5947, 3.2307],
          [2.4661, 3.2307, 2.3032]],
 
         [[2.0207, 1.9999, 2.3251],
          [1.9999, 2.3641, 2.5388],
          [2.3251, 2.5388, 2.8364]]]),
 'f': tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]])}

In [8]:
import numpy as np
import torch

from CANNLib.ANNLayers.EnergyLayers import *
from CANNLib.ANNLayers.StructuralLayers import *
from CANNLib.LambdaLayers.InvariantsLayers import *


class CANNClassic(torch.nn.Module):

  def __init__(self, J = 3, R = 5, f = 3, *args, **kwargs) -> None:
      super(CANNClassic,self).__init__(*args, **kwargs)

      self.DC = DirectionComputer(feature_number = f, J = J)
      self.WC = WeightComputer(feature_number = f, J = J, R = R)
      self.PC = PsiComputer(R = R)

      self.STC = StructuralTensorComputer()
      self.GSTC = GeneralizedStructuralTensorComputer()
      self.GIC = GeneralizedInvariantsComputer()

  def forward(self,data):

    C_ = data['C']
    f_ = data['f']


    l_ = self.DC(f_)
    w_ = self.WC(f_)

    #print(l_,w_)

    L_ = self.STC(l_)
    Lgen_ = self.GSTC(L_,w_)
    IJgen_ = self.GIC(Lgen_,C_)

    #print(L_,Lgen_,IJgen_)

    Psi_ = self.PC(IJgen_)

    return Psi_


In [9]:
model = CANNClassic()

In [10]:
model(X)

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)